# Check missings:  images, videos and annotations
version: 1

info: 


author: nuno costa

In [2]:
from annotate_v5 import *
import platform 
import numpy as np
import pandas as pd
from IPython.display import Image, display
import copy
import os
import random

In [35]:
dataset_root="TAO/new_TAO_DIR"

In [36]:
#Define root dir dependent on OS
rdir_dsets='/home/administrator/Z/Datasets/External Datasets/' #WARNING: DATASETS ROOT is OK?
rdir='/home/administrator/Z/Datasets/External Datasets/'+ dataset_root +'/'
print('OS: {}'.format(platform.platform()))
print('root datasets dir: {}'.format(rdir_dsets))
print('root dir: {}'.format(rdir))

OS: Linux-5.4.0-65-generic-x86_64-with-glibc2.10
root datasets dir: /home/administrator/Z/Datasets/External Datasets/
root dir: /home/administrator/Z/Datasets/External Datasets/TAO/new_TAO_DIR/


In [13]:
#jsonfile
injsonfile_rdir="annotations/train" #"split_mola_fix_equal/test"
molajson =  json.load(open(rdir+injsonfile_rdir+'.json'))
for k in molajson:
    print(k, len(molajson[k]))

videos 500
annotations 54639
tracks 2647
images 18274
info 6
categories 1230
licenses 1


# A. Verify using dataset toolkit

In [39]:
#TAO
if (dataset_root.upper().find("TAO")>-1):
    #https://github.com/TAO-Dataset/tao
    #https://github.com/TAO-Dataset/tao/blob/master/docs/download.md
    TAO_ROOT='"'+rdir+'"'#'"'+rdir+'"'
    TAO_SCRIPT='"/home/administrator/Z/Datasets/External Datasets/TAO/tao/scripts/download/verify.py"'
    PASSFILE="/home/administrator/Desktop/remote.txt"
    !sudo -S python $TAO_SCRIPT $TAO_ROOT --split train <$PASSFILE

[sudo] password for administrator:   File "/home/administrator/Z/Datasets/External Datasets/TAO/tao/scripts/download/verify.py", line 28
    ann_path = args.root / f'annotations/{args.split}.json'
                                                          ^
SyntaxError: invalid syntax


## 1. Import
#### #NOTE: work with ids and index so you can use numpy for faster operations

In [19]:
# datasets name and id
dset_l=[]
dset_l_id=[]
try:
    for d in molajson['datasets']:
        dset_l.append(d['name'])
        dset_l_id.append(d['id'])
except: #manually add for example for only COCO
    dset_l=['TAO']
    dset_l_id=[1]
print(dset_l, dset_l_id)

['TAO'] [1]


In [20]:
# categories name and id
cat_l=[]
cat_l_id=[]
cat_l_dset=[]
for c in molajson['categories']:
    cat_l.append(c['name'])
    cat_l_id.append(c['id'])
    try:
        cat_l_dset.append(dset_l[c['dataset']-1]) # dset_l index is same as id-1
    except:
        cat_l_dset.append(dset_l[0])
#print(cat_l_id)

In [21]:
# images filepath and id
img_l=[]
img_l_id=[]
for c in molajson['images']:
    img_l.append(c['file_name'])
    img_l_id.append(c['id'])

In [26]:
# videos filepath and id
try:
    vid_l=[]
    vid_l_id=[]
    for c in molajson['videos']:
        vid_l.append(c['name'])
        vid_l_id.append(c['id'])
except Exception as e:
    print(e)
    print("No video")

In [27]:
# tracks 
try:
    track_l_id=[]
    track_l_vidid=[]
    for c in molajson['tracks']:
        track_l_id.append(c['id'])
        track_l_vidid.append(c['video_id'])
except Exception as e:
    print(e)
    print("No video")

In [30]:
# annotations category_id, image_id, bbox, and dataset
ann_catid=[]
ann_imgid=[]
ann_bbox=[]
ann_dset=[]
ann_trackid=[]
for an in tqdm(molajson['annotations']):
    ann_catid.append(an['category_id'])
    ann_imgid.append(an['image_id'])
    ann_bbox.append(an['bbox'])
    try: 
        ann_trackid.append(an['track_id'])
    except:
        pass
    try:
        ann_dset.append(an['dataset'])
    except:
        ann_dset.append(dset_l_id[0])

100%|██████████| 54639/54639 [00:00<00:00, 480695.57it/s]


## 2. CHECK IMAGES

In [44]:
img_missing_l=[]
for img in tqdm(img_l):
    src = os.path.join(rdir, img)
    if (dataset_root.upper().find("TAO")>-1): src = os.path.join(rdir,"frames", img)
    if not os.path.exists(src): 
        img_missing_l.append(img)
print(src)
print(len(img_missing_l))

100%|██████████| 18274/18274 [00:03<00:00, 5789.20it/s]

/home/administrator/Z/Datasets/External Datasets/TAO/new_TAO_DIR/frames/train/Charades/8Q0EY/frame0931.jpg
6621


### 2.1 CHECK ANNOTATIONS (with missing images, that have good bounding box and are are not "iscrowd")

In [49]:
# Create image dict {id: image}
images = {'%g' % x['id']: x for x in molajson['images']}

# annotations category_id
new_annotations=[]
img_missing_l=[]
ann_missing_l=[]
for an in tqdm(molajson['annotations']):
    try:
        if an['iscrowd']: continue
    except:
        print('missing "iscrowd" key')
    img = images['%g' % an['image_id']]
    h, w, f = img['height'], img['width'], img['file_name']


    # The Labelbox bounding box format is [top left x, top left y, width, height]
    box = np.array(an['bbox'], dtype=np.float64)
    box[:2] += box[2:] / 2  # xy top-left corner to center
    box[[0, 2]] /= w  # normalize x
    box[[1, 3]] /= h  # normalize y

    if (box[2] > 0.) and (box[3] > 0.):  # if w > 0 and h > 0
        src = os.path.join(rdir, f)
        if (dataset_root.upper().find("TAO")>-1): src = os.path.join(rdir,"frames", f)
        if os.path.exists(src): new_annotations.append(an)
        else: 
            img_missing_l.append(img)
            ann_missing_l.append(an)
            

print(len(molajson['annotations']))
print(len(new_annotations))
print(len(img_missing_l))

100%|██████████| 54639/54639 [00:03<00:00, 13884.73it/s]

54639
32008
22631


### 2.2 CHECK Classes|categories (annotations and images for each class w/ EXCEL report) 

In [50]:
#checks #TODO: SORT alphabetically
checks_l=[]
checks_l_catid=[]
checks_l_catdset=[]
check_method="all_cats"
if check_method=="all_cats": #Do for all category names, even with equal 
    checks_l=cat_l
    checks_l_catid=[[id] for id in cat_l_id]
    checks_l_catdset=[[dset] for dset in cat_l_dset]
    

print(checks_l[0:5])
print(checks_l_catid[0:5])
print(checks_l_catdset[0:5])
print(len(checks_l))
print(len(checks_l_catid))
print(len(checks_l_catdset))

['acorn', 'aerosol_can', 'air_conditioner', 'airplane', 'alarm_clock']
[[1], [2], [3], [4], [5]]
[['TAO'], ['TAO'], ['TAO'], ['TAO'], ['TAO']]
1230
1230
1230


In [51]:
# get annotations checks
ann_catid_np=np.array(ann_catid)
ann_imgid_np=np.array(ann_imgid)
ann_bbox_np=np.array(ann_bbox)
ann_dset_np=np.array(ann_dset)
checks_l_imgid=[]
checks_l_bbox=[]
checks_l_dset=[]
for catids in tqdm(checks_l_catid):
    l_imgid=[]
    l_bbox=[]
    l_dset=[]
    for catid in catids:
        ann_idx = np.where(ann_catid_np==catid)[0].tolist() #annotation index of ids
        l_imgid.append(ann_imgid_np[ann_idx].tolist())
        l_bbox.append(ann_bbox_np[ann_idx].tolist())
        l_dset.append(ann_dset_np[ann_idx].tolist())
    checks_l_imgid.append(l_imgid)
    checks_l_bbox.append(l_bbox)
    checks_l_dset.append(l_dset)

100%|██████████| 1230/1230 [00:00<00:00, 17485.21it/s]


In [52]:
#INIT VARS
method="save_images" #"save_images": to save new images and excel for manual inspection; "": already saved don't need to repeat and the excel are
datadir="checks/"+injsonfile_rdir+"/" #root folder to save check method . #WARNING checks/original json that was used to save images and excel
folder=check_method+'/' #folder to save images and exel 
showimage=False #show images
startidx=0 # start index of image to save from each dataset
imgnr=3 # total number of images to save from each dataset
imgstep='random' # step between images: int | 'random' - int steps between images; 'rand' gets random list
#paths
path=os.path.join(rdir,datadir,folder) #path to folder
assure_path_exists(path)
excelpath=path+check_method+"_report_v1.xlsx"#path+check_method+"_report.xlsx"#path to excel

In [ ]:
#METHODS
if method=="save_images": # save images and excel to folder for manual edit
    df=pd.DataFrame({'checks_l': checks_l,'checks_l_catid': checks_l_catid, 'checks_l_catdset': checks_l_catdset })
    df['annotations_missing'] = np.empty((len(df), 0)).tolist()
    df['images_missing'] = np.empty((len(df), 0)).tolist()
    #save image for each check
    for i, check in enumerate(tqdm(checks_l)): #run for each check category
        firstclass=check
        if isinstance(firstclass, list): firstclass=firstclass[0] #first class
        print('\n>> '+firstclass+'...') #class
        classpath=os.path.join(path, firstclass) # path to folder for images of  firstclass
        classpath=parse_path(classpath)+'/' #make it a folder
        assure_path_exists(classpath)
        rdir_path=rdir_dsets
        if (dataset_root.upper().find("TAO")>-1): rdir_path = os.path.join(rdir,"frames/")
        df=save_imgs(df, rdir_path, classpath, i, dset_l, checks_l, checks_l_catid, checks_l_bbox, checks_l_dset,
              checks_l_imgid, img_l, img_l_id, startidx=startidx, imgnr=imgnr, imgstep=imgstep, showimage=showimage)    
    df.to_excel(excelpath, index=False)   

In [56]:
#IMPORT EXCEL MANUAL EDIT #WARNING: CHECK EXCEL FIRST (#NOTE: donte use classes with missing annotations and images)
df=pd.read_excel(excelpath)
display(df)        

,checks_l,checks_l_catid,checks_l_catdset,annotations_missing,images_missing
0,acorn,[1],['TAO'],[1],[0]
1,aerosol_can,[2],['TAO'],[0],[1]
2,air_conditioner,[3],['TAO'],[1],[0]
3,airplane,[4],['TAO'],[0],[0]
4,alarm_clock,[5],['TAO'],[1],[0]
...,...,...,...,...,...
1225,yak,[1226],['TAO'],[1],[0]
1226,yogurt,[1227],['TAO'],[1],[0]
1227,yoke_(animal_equipment),[1228],['TAO'],[1],[0]
1228,zebra,[1229],['TAO'],[0],[0]


## 3. CHECK VIDEOS

In [57]:
vid_missing_l=[]
for vid in vid_l:
    src = os.path.join(rdir, vid)
    if (dataset_root.upper().find("TAO")>-1): src = os.path.join(rdir,"videos", vid)
    if not os.path.exists(src): 
        vid_missing_l.append(vid)
print(src)
print(len(vid_missing_l))

/home/administrator/Z/Datasets/External Datasets/TAO/new_TAO_DIR/videos/train/Charades/8Q0EY
500


### 3.1 CHECK ANNOTATIONS (with missing ivideos)

In [ ]:
# Create image dict {id: image}
videos = {'%g' % x['id']: x for x in molajson['videos']}
tracks = {'%g' % x['id']: x for x in molajson['tracks']}
# annotations category_id
new_annotations=[]
vid_missing_l=[]
ann_missing_l=[]
for an in tqdm(molajson['annotations']):
    try:
        if an['iscrowd']: continue
    except:
        print('missing "iscrowd" key')
    
    vid = videos['%g' % tracks['%g' % an['track_id']]]
    h, w, f = vid['height'], vid['width'], vid['name']


    # The Labelbox bounding box format is [top left x, top left y, width, height]
    box = np.array(an['bbox'], dtype=np.float64)
    box[:2] += box[2:] / 2  # xy top-left corner to center
    box[[0, 2]] /= w  # normalize x
    box[[1, 3]] /= h  # normalize y

    if (box[2] > 0.) and (box[3] > 0.):  # if w > 0 and h > 0
        src = os.path.join(rdir, f)
        if (dataset_root.upper().find("TAO")>-1): src = os.path.join(rdir,"videos", f)
        if os.path.exists(src): new_annotations.append(an)
        else: 
            vid_missing_l.append(img)
            ann_missing_l.append(an)
            

print(len(molajson['annotations']))
print(len(new_annotations))
print(len(vid_missing_l))